# Experimentos algorítmicos típicos

Suponer que tengo la posibilidad de ejecutar un experimento equivalente a un bit de probabilidad p

In [111]:
from random import choice
from fractions import Fraction as F

def bit(p):
    p = F(str(p)) # truquito
    return choice([1] * p.numerator + [0] * (p.denominator - p.numerator))

In [112]:
[bit(0.3) for _ in range(10)]

[1, 0, 0, 1, 0, 1, 0, 0, 1, 0]

## Repetirlo n veces incondicionalmente

También conocido como _Binomial_.

In [113]:
def repetirlo(n, p):
    secuencia = []
    for i in range(n):
        secuencia.append(bit(p))
    return secuencia

In [114]:
[repetirlo(3, 0.1) for _ in range(10)]

[[0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 1, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [1, 0, 0],
 [0, 0, 0]]

In [70]:
from itertools import product
n = 3
Omega = list(product(* ((0, 1),)*n))
Omega

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

¿Cuánto vale la probabilidad de que salga al menos un 1? Se puede obtener por complemento de "que salgan todos ceros":

$1 - (1 - p)^n$

Para esto se necesita introducir la noción de Evento

# Eventos

(predicados, álgebra de eventos, sí tiene estructura)

Son todas las posibles preguntas al experimento. Se puende pensar como proposiciones lógicas que se evalúan para cada resultado posible. Esto es equivalente a definir un subconjunto de elementos que "cumplen" con la proposición.

In [135]:
def al_menos_un_uno(w):
    return sum(w) > 0

In [137]:
sum(map(al_menos_un_uno, Omega)) / len(Omega)

0.875

In [148]:
M = 1000
n = 10
p = 0.1
len(list(filter(al_menos_un_uno, [repetirlo(n, p) for _ in range(M)]))) / M

0.654

In [149]:
1 - (1 - p)**n

0.6513215599

## Repetirlo hasta que sea 1

También conocido como _Geométrico_.

In [121]:
def repetir_hasta_que_suceda(p):
    secuencia = []
    while not (secuencia := secuencia + [bit(p)])[-1]:
        continue
    return secuencia

In [122]:
[repetir_hasta_que_suceda(0.3) for _ in range(10)]

[[0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 1],
 [0, 0, 0, 1],
 [1],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 1]]

Claramente, $\Omega$ contiene una cantidad infinita de elementos, pero por suerte es numerable e isomórfico con el conjunto de los números naturales $\mathbb{N}$. Estos son sus primeros 10 elementos:

In [133]:
sub_Omega = [[0] * k + [1] for k in range(10)] + ['...']; sub_Omega

[[1],
 [0, 1],
 [0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 '...']

### ¿Cómo asignar probabilidades?

Una manera de pensarlo es a partir de un experimento parecido: "tirar la moneda n veces" y para ese contexto, calcular la probabilidad de obtener la secuencia:

In [118]:
n = 5
[0] * (n - 1) + [1]

[0, 0, 0, 0, 1]

Se puede ver que la probabilidad es $(1 - p)^(n-1) \cdot p$. Podríamos probar asignar estas probabilidades a nuestro $\Omega$.

Podemos verificarlo usando frecuentismo:

In [123]:
from collections import Counter
M = 1000 # ejecuciones
p = 0.3
histograma = Counter(map(len, [repetir_hasta_que_suceda(p) for _ in range(M)]))
longitudes = sorted(histograma.keys())
print('longitud\tempírica\tpropuesta')
for l in longitudes:
    veces = histograma[l]
    print(f'{l:8}\t{veces/M:3f}\t{p*(1-p)**(l-1):3f}')

longitud	empírica	propuesta
       1	0.297000	0.300000
       2	0.220000	0.210000
       3	0.144000	0.147000
       4	0.116000	0.102900
       5	0.067000	0.072030
       6	0.043000	0.050421
       7	0.035000	0.035295
       8	0.025000	0.024706
       9	0.022000	0.017294
      10	0.008000	0.012106
      11	0.003000	0.008474
      12	0.009000	0.005932
      13	0.004000	0.004152
      14	0.001000	0.002907
      15	0.001000	0.002035
      16	0.003000	0.001424
      17	0.001000	0.000997
      18	0.001000	0.000698


In [104]:
from math import comb

In [116]:
comb(20, 6)

38760

In [134]:
bit = (0, 1)
p = 0.1
from itertools import product
n = 10
#sum(1 for e in product(*(bit, )*n) if sum(e) == 6)

In [135]:
evento = filter(lambda e: sum(e) == 4, product(*((bit,)*n)))

In [136]:
sum(map(lambda e: p**sum(e) * (1-p)**(n-sum(e)), evento))

0.011160260999999986

# ¿Cómo ajustar escalas de tiempo correctamente?

Suponer que uno sabe que la probabilidad de que un evento suceda "al menos una vez" durante una serie de n repeticiones es $p_n$,
¿qué podemos decir de la probabilidad de que suceda en la próxima repetición? (asumiendo uniformidad y simetría temporal)

Notar que **NO** estamos preguntando:
* ¿cuál es la probabilidad de que pase ahora sabiendo que SÍ o SÍ va a pasar en los próximos n?
* ¿cuál es la probabilidad de que pase ahora sabiendo que va a pasar UNA y sólo UNA vez en los próximos n?


In [162]:
(p_anio := 1 / 1428)

0.0007002801120448179

In [161]:
(p_dia := 1 - (1 - p_anio)**(1/365))

1.919245891657262e-06

In [160]:
(p_200dias := 1 - (1 - p_dia)**200)

0.00038377588587035216